## Process data

In [25]:
import pandas as pd

df = pd.read_pickle("processed_df.pkl")

<IPython.core.display.Javascript object>

## Terms

In [26]:
import re
import scattertext as st

<IPython.core.display.Javascript object>

In [27]:
analyze_col = "tasks"

<IPython.core.display.Javascript object>

In [28]:
filtered_df = df[~df[analyze_col].isna()][["title", analyze_col, "page_host"]]

<IPython.core.display.Javascript object>

In [29]:
filtered_df["parse"] = filtered_df[analyze_col].apply(st.whitespace_nlp_with_sentences)

<IPython.core.display.Javascript object>

In [30]:
corpus = (
    st.CorpusFromParsedDocuments(filtered_df, category_col="title", parsed_col="parse")
    .build()
    .get_unigram_corpus()
    .compact(st.AssociationCompactor(2000))
)

<IPython.core.display.Javascript object>

In [31]:
html = st.produce_scattertext_explorer(
    corpus,
    category="data scientist",
    category_name="Data scientist",
    not_category_name="Data Engineer",
    minimum_term_frequency=5,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    metadata=corpus.get_df()["page_host"],
    transform=st.Scalers.dense_rank,
)
open("data_science_vs_data_engineer_task_terms.html", "w").write(html)

1805457

<IPython.core.display.Javascript object>